<a href="https://colab.research.google.com/github/FadelYang/craft-it-object-detection-model/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing required Libraries
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from google.colab import files

# Preparing Dataset

## Mount to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


## Load Dataset



In [4]:
# !wget https://app.roboflow.com/ds/gzGSPSpsan?key=CrzyeSBgrQ
# !mv gzGSPSpsan?key=CrzyeSBgrQ data
# !mkdir dataset
# !unzip data -d dataset

In [5]:
## Load label dataset.
dataset_dir = os.path.join(os.getcwd(), 'dataset')
train_label_df = pd.read_csv(os.path.join(dataset_dir, 'train', '_classes.csv'))
validation_label_df = pd.read_csv(os.path.join(dataset_dir, 'valid', '_classes.csv'))
train_images_dir = os.path.join(dataset_dir, 'train')
validation_images_dir = os.path.join(dataset_dir, 'valid')

print(train_images_dir)

/content/drive/MyDrive/Colab Notebooks/dataset/train


In [6]:
print(train_label_df.head())
print(validation_label_df.head())

                                           filename   aluminiumfoil   bottle  \
0  1196_jpg.rf.68689b8efc68113f5ffa321d8c02fe0b.jpg               0        0   
1   757_jpg.rf.67b5d7ce20e152ea0f2b8783ffe65301.jpg               0        0   
2   344_jpg.rf.6725ba8501f47157d433edc3e1420a85.jpg               0        1   
3    65_jpg.rf.65d079f8e8e3840c836b3902febc65c8.jpg               0        0   
4   414_jpg.rf.665baa402c9f84c0416955ebe0ad3925.jpg               0        0   

    bottlecap   can   carton   cup   paper   plastic   straw  
0           0     0        0     0       0         0       1  
1           0     0        0     1       0         0       0  
2           0     1        0     0       0         0       0  
3           1     0        0     0       0         0       0  
4           0     1        0     0       0         0       0  
                                          filename   aluminiumfoil   bottle  \
0  121_jpg.rf.030d2a2ac6f40bab2e9bea71c50b7716.jpg            

In [ ]:
# !rm -r train

In [7]:
train_label_df.columns = train_label_df.columns.str.strip()
validation_label_df.columns = validation_label_df.columns.str.strip()

In [8]:
# # Check the data.
# import os

# train_image_filenames = train_label_df['filename'].tolist()
# validation_image_filenames = validation_label_df['filename'].tolist()

# train_image_paths = [os.path.join(train_images_dir, fname) for fname in train_image_filenames]
# validation_image_paths = [os.path.join(validation_images_dir, fname) for fname in validation_image_filenames]

# # Check if all files exist
# missing_train_images = [img for img in train_image_paths if not os.path.exists(img)]
# missing_validation_images = [img for img in validation_image_paths if not os.path.exists(img)]

# print(f"Missing train images: {missing_train_images}")
# print(f"Missing validation images: {missing_validation_images}")

In [9]:
columns = [
    "aluminiumfoil",
    "bottle",
    "bottlecap",
    "can",
    "carton",
    "cup",
    "paper",
    "plastic",
    "straw"
]

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)


if os.path.exists(train_images_dir):
  train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_label_df,
      directory=train_images_dir,
      x_col='filename',
      y_col=columns,
      batch_size=32,
      seed=42,
      class_mode='raw',
      shuffle=True,
)

if os.path.exists(validation_images_dir):
  validation_generator = validation_datagen.flow_from_dataframe(
      dataframe=validation_label_df,
      directory=validation_images_dir,
      x_col='filename',
      y_col=columns,
      batch_size=32,
      class_mode='raw',
      seed=42,
      shuffle=False,
)

Found 2331 validated image filenames.
Found 227 validated image filenames.


# Create Model and Training

## Build Model



In [15]:
resnet_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=tf.keras.Input(shape=(640, 480, 3))
)

In [16]:
# Freeze layers in ResNet50
resnet_model.trainable = False

print(resnet_model)
print(train_generator)

In [33]:

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(num_classes, activation='sigmoid')(x)
# model = Model(inputs=base_model.input, outputs=predictions)

# Add dense layers for classification
x = resnet_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
outputs = tf.keras.layers.Dense(9, activation='sigmoid')(x)

# Create the model using the functional API
model = tf.keras.Model(inputs=resnet_model.input, outputs=outputs)

In [32]:
x = resnet_model.output.shape
print(x)

(None, 20, 15, 2048)


In [34]:
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/25
12/72 [====>.........................] - ETA: 9:35 - loss: 0.4580 - accuracy: 0.2109 